In [1]:
#importing neccessary packages
import string#in built class string for string related operations
import sys#in  built class sys to check system arguments given
import os
import sys
import re
import glob
import codecs
import math
from operator import itemgetter

In [3]:
print("የሚፈልጉትን ጥያቄ ካስገቡ ቡኋላ እባከዎ ENTER ይበሉና ጥቂት ደቂቃ ይታገሱ ！ከዘገይበዎት በደጋሚ ENTER ይበሉ :-")

class Amharic_search():

    def indexer(path): 
        def pucrmv(str):#defining a function that helps to remove punctuatuion
            str=re.sub('[.''፡፡''!፡()"፣?/!።]','',str)#regular expration is subdivided in to sub string
            if str.endswith('\n'):#check wheather the given string is ends with  new line character or not
                str=str.rstrip('\n')#removes  \n characters from right
            if str.startswith(' '):#check is the given string is starts with space
                str=str.lstrip(' ')#removes space characters frrom the left
            s=codecs.open("Am_pun.txt",'r', encoding = 'utf-8' )#Open an encoded file using the given mode and 
            #codecs return an instance of StreamReaderWriter , providing transparent encoding/decoding.
            puc = s.read()#read punctuations from the text file
            for pc in puc:#iterate through all the marks
                if str.endswith(pc):#chcke is the given string ends with pppunctuation
                    str=str.rstrip(pc)#remove it
                if str.startswith(pc):#chcke is the given string start with pppunctuation
                    str=str.lstrip(pc)#remove it
            return str#return the string after removing punctuation marks
        def normal(str):#defining function for normalization; which means treating leters that has the same phonems in simiar 

            let=codecs.open("Am_nor.txt",'r', encoding = 'utf-8' )#open text file that has normals
            nor = let.read()#read each letters
            li=[]#define a list that used to contain hohies that useful for normalization
            for j in nor.split():
                li.append(j)
                
            for i in range(0,len(li)-1,2):
                str=str.replace(li[i],li[i+1])
            return str

        def stem(word):#hfunction for finding the stem word; which means by removing the prefix orsuffix
            s=codecs.open("Am_suf.txt",'r', encoding = 'utf-8' )
            su = s.read()
            p=codecs.open("Am_pre.txt",'r', encoding = 'utf-8' )
            pr = p.read()#read the prefix list from text file

            for suf in su.split():#loop or iterate through the suffix term list
                if len(word)>2:#check the word length 
                    
                    if word.endswith(suf):#check the word is ends with suffix list or not
                           word=word [ :-len(suf) ]#remove the suffix from the word and re assign the word
            for pre in pr.split():
                if len(word)>2:
                    if word.startswith(pre):
                        l=len(pre)
                        word=word[l:]
            word=normal(word)
            return word
        def readfile(filename):#for reading the file
            indexterms=[]
            infile=codecs.open(filename, encoding='utf-8')
            lterms=infile.readlines()
            infile.close()
            for str in lterms:
                if str.endswith('\n'):
                    str=str.rstrip('\n')
                indexterms.append(str)
            return indexterms 
        def doclister(pst):
            ldocid=[]
            infile=open(pst)
            lterms=infile.readlines()
            infile.close()
            for str in lterms:
                if str.endswith('\n'):
                    str=str.rstrip('\n')
                ldocid.append(str)
            return ldocid
        def iterm(ldocid):
            fileterms=[]
            
            indexterms=[]
            indexterm=[]
            stopw=readfile('Am_sto.txt')
            for docid in ldocid:
                fileterms=readfile(docid)
                for listterm in fileterms:
                    for index in listterm.split():
                        flag=0
                        for i in range(0,len(stopw)):
                            if stem(pucrmv(index))in stopw[i]:
                                flag=1
                        if flag==0:             
                            indexterms.append(stem(pucrmv(index)))
            for it in range(0,len(indexterms)):
                flag=0
                for jj in range(0,len(indexterm)):
                    if indexterms[it] == indexterm[jj]:
                        flag=1
                if flag==0:
                    indexterm.append(indexterms[it])
                    #print("term=",indexterm)
            #for i in range(0,len(indexterm)):
                #print (indexterm[i])
            return indexterm
    
        def termfreq(indexterm,ldocid):
            f=0
            cf=0
            tfiles=[]
            var=0
            dicindex=indexterm
            for index in indexterm:
                dicindex[var]={}
                for doc in ldocid:
                    tfiles=readfile(doc)
                    f=0
                    for str in tfiles: 
                        for term in str.split(" "):
                            if stem(pucrmv(term))== index:
                                f=f+1                                
                    dicindex[var][doc]=f#stores the frequncy of each index term in each documnet in dicindex dictionary
                var=var+1# increments index of each index term
            
            return dicindex
        def userquery(indexlist, dicindex, ldocid):#the user query accepts those variable and return
            
            lquery=[]
            iquery=[]
            dicquery={}
            stopw=readfile('Am_sto.txt')
            query=input("የሚፈልጉትን ጥያቄ ያስገቡ")
            query=query.lstrip()
            query=query.rstrip()
            qlength=len(query.split(" "))
            if len(query)>0:
                if len(query)>=1:
                    if qlength<15:
                        # To remove stopword from the user query, to stem user query word and append the user query word in a list called lquery
                        for qin in query.split(" "):
                            flag=0
                            for i in range(0,len(stopw)):
                                if stem(pucrmv(qin))in stopw[i]:
                                    flag=1
                            if flag==0:
                                lquery.append(stem(pucrmv(qin)))
                                
                        # To identify unique word from the user query and append them in a list called iquery
                        idx=0
                        for iq in lquery:
                            if not iquery.__contains__(iq):
                                if not iq =='':
                                    iquery.append(iq)
                            # To check the term whether it is in the corpus or not.
                            #If it is not found then for error handling the term will be append in the indexterm,
                            #and the dictionery(dicindex) that contain the stastical infromation about the term will be updated
                            if not indexlist.__contains__(iq):
                                indexlist.append(iq)
                                idx=indexlist.index(iq)
                                dicindex.append({})
                                for doc in ldocid:
                                    dicindex[idx][doc]=0

                        # To callculate the term frequency of the user query and append them in a dictionary called dicquery
                        for iq in iquery:
                            f=0
                            for term in lquery:
                                if term == iq:
                                    f=f+1                    
                            dicquery[iq]=f

                        #To identify the maximum term frequency of a document and append on a dictionary called maxfdoc
                        #and this will be used to normalize the termfrequency
                        maxfdoc={}
                        ndoc=len(ldocid)
                        for doc in ldocid:
                            maxf=0
                            for i in range(0,len(indexlist)):#changed 
                                if maxf<dicindex[i][doc]:
                                    maxf=dicindex[i][doc]
                            maxfdoc[doc]=maxf

                        #TO calculate document frequency for each indexterm and append it in a dictionary called dicndci
                        #it will be used to callculate inverse document frequency
                        dicndci={}
                        for iq in range(0,len(indexlist)):
                            ndci=0
                            tindex=indexlist.index(indexlist[iq])
                            for ld in ldocid:
                                if dicindex[tindex][ld]>0:
                                    ndci=ndci+1
                            dicndci[indexlist[iq]]=ndci

                        #To calculate similarity between each documents and query using cosine similarity
                        #the term weitght here is that it uses tfidf
                        #and it append the document and its similarity value in the a dictionary called diccos
                        diccos={}
                        for doc in ldocid:
                            upv=0
                            lvq=0
                            lvd=0
                            lv=0
                            value=0.0           
                            for iq in indexlist:
                                tindex=indexlist.index(iq)
                                #to calculate the normalized term frequency for the indexterm iq
                                tf=(dicindex[tindex][doc]+0.0)/maxfdoc[doc]
                                if not dicndci[iq]==0:
                                    #to calculate the inverse document frequency for the indexterm iq
                                    idf=math.log(((len(ldocid)+0.0)/dicndci[iq]),2)
                                else:
                                    idf=0
                                tfidf=tf*idf
                                #if the indexterm is found in the query
                                if iquery.__contains__(iq):
                                    upv=upv+dicquery[iq]*tfidf
                                    lvq=lvq+pow(dicquery[iq],2)
                                lvd=lvd+pow(tfidf,2)
                            lv=math.sqrt((lvq*lvd))
                            if not lv ==0:
                                value=upv/lv
                                diccos[doc]=value

                        ranklist=[]
                        ranklist=list(diccos.keys())

                        #to check whether there is a relevant document or not
                        flag=0
                        for i in range(0,len(ranklist)):
                            if diccos[ranklist[i]] !=0:
                                flag=1
                        #to rank the similarity of document with the query in decreasing order
                        readf=[]
                        for i in range(len(ranklist)):
                            for j in range(i+1,len(ranklist)):
                                if diccos[ranklist[i]]<diccos[ranklist[j]]:
                                    temp=ranklist[i]
                                    ranklist[i]=ranklist[j]
                                    ranklist[j]=temp
                        if flag==1:
                            print("**********************************************************************")
                            print ("የተገኙት መዛግብት ባላቸው ቅርበት መሰረት እንደሚክተለው ተቀምጠዋል። ")
                            print("**********************************************************************")
                            print ("ደረጃ\tየመረጃ ስም \tየመመሳሰል ቅርበት")
                            print ("---------   ---------------       -----------------------------")
                            for i in range(len(ranklist)):
                                if diccos[ranklist[i]]>0:
                                   print (i+1,"\t",ranklist[i][7:],"\t\t",diccos[ranklist[i]])
                                   readf.append(ranklist[i][7:])
                            print ("______________________________________________")
                            read=input("   ማንበብ ይፈልጋሉ ?  የመረጃዉን ስም ተጠቅምዉ ይምረጡ ።ያለበለዚያ ሌላ ፊደል ይጫኑ")
                            content=""
                            if read in readf:
                                print("መረጃዉ እንደሚከተለዉይ ንበባል")
                                print ("_____________________________________________________________________________")
                                infile=codecs.open('corpus/'+read,'r',encoding = 'utf-8')
                                content=infile.read()
                                
                                print(content)
                                readds=input("   እንደግና ፍለጋ ለመሞከር ይፈልጋሉ? ከፈለጉ Y ቁጥርን ይጫኑ፤ ያለበለዚያ ሌላ ፊደል ይጫኑ።")
                                if readds=='Y':
                                      userquery(indexlist, dicindex, ldocid)
                                else:
                                      print ("                           ")
                                      print ("\t እናመሰግናለን!  እንደገና ይጎብኙን!")   
                            else:
                                print("   ይቅርታ ያስገቡት መረጃ በዝርዝሩ ዉስጥ የለም ።")
                                print ("_____________________________________________________________________________")
                                readd=input("   እንደገና ፍለጋ ለመሞከር ይፈልጋሉ? ከፈለጉ 1 ቁጥርን ይጫኑ፤ ያለበለዚያ ሌላ ፊደል ይጫኑ።")
                                if readd=='1':
                                    userquery(indexlist, dicindex, ldocid)
                                else:
                                    print ("                           ")
                                    print ("\t እናመሰግናለን!  እንደገና ይጎብኙን!")
                        else:
                            print ("ይቅርታ! ከጠየቁት መጠይቅ ጋር የሚመሳሰል መረጃ አልተገኘም።")
                            print ("_____________________________________________________________________________")
                            readd=input("   እንደግና ፍለጋ ለመሞከር ይፈልጋሉ? ከፈለጉ 1 ቁጥርን ይጫኑ፤ ያለበለዚያ ሌላ ፊደል ይጫኑ።")
                            if readd=='1':
                                  userquery(indexlist, dicindex, ldocid)
                            else:
                                  print ("                           ")
                                  print ("\t እናመሰግናለን!  እንደገና ይጎብኙን!")    
                            
                    else:
                        print ("እባክዎ ከ 15 ያነሱ ቃላትን ይጠቀሙ።")
                        userquery(indexlist, dicindex, ldocid)
                else:
                    print ("ያስገቡት መጠይቅ ተፈላጊ የሆኑ መረጃዎችን ለማግኘት አያስችልም።")
                    userquery(indexlist, dicindex, ldocid)
            else:
               print( "\tምንም ዓይነት መጠይቅ አላስገቡም፤ እባክዎ እንደገና ይሞክሩ\n")
               userquery(indexlist, dicindex, ldocid)
               
       
        return userquery(iterm(doclister(path)), termfreq(iterm(doclister(path)),doclister(path)),doclister(path))

   
path="Document_identifier.txt"
Amharic_search.indexer(path)


    
        



የሚፈልጉትን ጥያቄ ካስገቡ ቡኋላ እባከዎ ENTER ይበሉና ጥቂት ደቂቃ ይታገሱ ！ከዘገይበዎት በደጋሚ ENTER ይበሉ :-
የሚፈልጉትን ጥያቄ ያስገቡየባህርዳር ዩኒቨርሲቲ ተማሪ
**********************************************************************
የተገኙት መዛግብት ባላቸው ቅርበት መሰረት እንደሚክተለው ተቀምጠዋል። 
**********************************************************************
ደረጃ	የመረጃ ስም 	የመመሳሰል ቅርበት
---------   ---------------       -----------------------------
1 	 Doc15.txt 		 0.2343622856774771
2 	 Doc17.txt 		 0.21723923643753654
3 	 Doc8.txt 		 0.20005374784437183
4 	 Doc12.txt 		 0.16714377865112756
5 	 Doc11.txt 		 0.1415564057871132
6 	 Doc2.txt 		 0.11210966198842147
7 	 Doc4.txt 		 0.10510637953326624
8 	 Doc7.txt 		 0.08394918091520517
9 	 Doc1.txt 		 0.07577179005512108
10 	 Doc9.txt 		 0.05941820509070592
______________________________________________
   ማንበብ ይፈልጋሉ ?  የመረጃዉን ስም ተጠቅምዉ ይምረጡ ።ያለበለዚያ ሌላ ፊደል ይጫኑDoc15.txt
መረጃዉ እንደሚከተለዉይ ንበባል
_____________________________________________________________________________
﻿ባህርዳር ዩኒቨርሲቲ አምስተኛውን የትምህርት ጥራት ኮንፈረንስ